In [1]:
import numpy as np

from astropy.io import fits
from astropy import units as u

%config InlineBackend.figure_format = 'retina'
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import summary_stat

In [3]:
# read the EL-COSMOS catalog
cmc = fits.open('/Volumes/ThunderBay4-40TB/CosmoTreasure/mock/cosmos/cmc_lam/combinedcatalog_flux_SNR-cos_full_phy_fz-a0.20_OI15config_added-Re.fits')[1].data
    
# specify a selection in (g, g-r, r-i): here is modified version of Takada+(2014) (see the previous SSP document)
select_g  = (cmc['g_hsc'] > 23.2) & (cmc['g_hsc'] < 24.2)
select_gr = ((cmc['g_hsc']-cmc['r_hsc'] > 0.05) 
                 & (cmc['g_hsc']-cmc['r_hsc'] < 0.35))
select_ri = ((cmc['g_hsc'] > 23.6) 
                 & (cmc['r_hsc']-cmc['i_hsc'] > 0.3))
select_g_gr = select_g & select_gr
cut_target  = select_g_gr & ~select_ri
    
numELGs, pfs_mT14_sn6, fac_eff = summary_stat.calc_target_stats(cmc, cut_target, snr_threshold=6, verbose=1)
dndz_mT14 = summary_stat.calc_dndz(pfs_mT14_sn6, fac_eff, verbose=1)

***** Target selection statistics summary *****

Your threthold: SNR_PFS_OII > 6

(1) # of fibers for which a galaxy is assigned: 4410
# of fibers which miss a galaxy assignment: 378
efficiency = (1)/(# of total fibers, 4788): 92.11%

(2) # of target galaxies per FoV: 8530
completeness = (1)/(2): 51.70%

(3) # of detectable PFS ELGs per two visits per FoV: 3052
success rate = (3)/(1) = 69.21%

***********************************************

dn/dz: 
#0, 0.6 <= z < 0.8, Vs[(Gpc/h)^3] = 0.550, Ng/FoV = 95, ng[10^-4(h/Mpc)^3] = 2.304
#1, 0.8 <= z < 1.0, Vs[(Gpc/h)^3] = 0.723, Ng/FoV = 390, ng[10^-4(h/Mpc)^3] = 7.192
#2, 1.0 <= z < 1.2, Vs[(Gpc/h)^3] = 0.864, Ng/FoV = 387, ng[10^-4(h/Mpc)^3] = 5.972
#3, 1.2 <= z < 1.4, Vs[(Gpc/h)^3] = 0.973, Ng/FoV = 731, ng[10^-4(h/Mpc)^3] = 10.022
#4, 1.4 <= z < 1.6, Vs[(Gpc/h)^3] = 1.052, Ng/FoV = 489, ng[10^-4(h/Mpc)^3] = 6.195
#5, 1.6 <= z < 2.0, Vs[(Gpc/h)^3] = 2.254, Ng/FoV = 711, ng[10^-4(h/Mpc)^3] = 4.206
#6, 2.0 <= z < 2.4, Vs[(Gpc/h)^3] = 2.345,

In [7]:
# can try a different criteria

# For example, DESI FDR
snr_threshold = 8E-17 #[erg/s/cm2]
snr_label = 'flux_OII'
zlow  = 0.6
zhigh = 1.6

select_r  = cmc['r_hsc'] < 23.4
select_A  = cmc['r_hsc'] - cmc['z_hsc'] > 0.3
select_rz  = cmc['r_hsc'] - cmc['z_hsc'] < 1.6
select_B  = cmc['g_hsc'] - cmc['r_hsc'] < 1.5*(cmc['r_hsc'] - cmc['z_hsc']) - 0.15
select_C  = cmc['g_hsc'] - cmc['r_hsc'] < 1.6 - 1.2*(cmc['r_hsc'] - cmc['z_hsc'])

select_FDRlike = select_r & select_A
select_FDRlike &= select_rz
select_FDRlike &= select_B
select_FDRlike &= select_C

dum_numELGs, dum_cat, fac_eff = summary_stat.calc_target_stats(cmc, select_FDRlike, snr_threshold, snr_label, zlow, zhigh, verbose=1)
dndz_dum = summary_stat.calc_dndz(dum_cat, fac_eff, verbose=1)

***** Target selection statistics summary *****

Your threthold: flux_OII > 8e-17

(1) # of fibers for which a galaxy is assigned: 4272
# of fibers which miss a galaxy assignment: 516
efficiency = (1)/(# of total fibers, 4788): 89.24%

(2) # of target galaxies per FoV: 7616
completeness = (1)/(2): 56.09%

(3) # of detectable PFS ELGs per two visits per FoV: 3248
success rate = (3)/(1) = 76.03%

***********************************************

dn/dz: 
#0, 0.6 <= z < 0.8, Vs[(Gpc/h)^3] = 0.550, Ng/FoV = 1183, ng[10^-4(h/Mpc)^3] = 28.690
#1, 0.8 <= z < 1.0, Vs[(Gpc/h)^3] = 0.723, Ng/FoV = 896, ng[10^-4(h/Mpc)^3] = 16.524
#2, 1.0 <= z < 1.2, Vs[(Gpc/h)^3] = 0.864, Ng/FoV = 514, ng[10^-4(h/Mpc)^3] = 7.932
#3, 1.2 <= z < 1.4, Vs[(Gpc/h)^3] = 0.973, Ng/FoV = 405, ng[10^-4(h/Mpc)^3] = 5.552
#4, 1.4 <= z < 1.6, Vs[(Gpc/h)^3] = 1.052, Ng/FoV = 248, ng[10^-4(h/Mpc)^3] = 3.142
#5, 1.6 <= z < 2.0, Vs[(Gpc/h)^3] = 2.254, Ng/FoV = 0, ng[10^-4(h/Mpc)^3] = 0.000
#6, 2.0 <= z < 2.4, Vs[(Gpc/h)^3] = 2.34